# Исследование уровня потребительской лояльности в телекоммуникационной компании

## Описание проекта

В этом учебном проекте необходимо построить дашборд для оценки уровня потребительской лояльности (NPS) в телекоммуникационной компании после проведенного опроса, а также ответить на вопросы с помощью этого дашборда и презентовать результаты в презентации.  
Данные опроса были выгружены в SQLite. Необходимо выгрузить эти данные в виде таблицы, состоящей из полей:
- `user_id` - Идентификатор клиента;
- `lt_day` - Количество дней «жизни» клиента;
- `is_new` - Поле хранит информацию о том, является ли клиент новым;
- `age` - Возраст;
- `gender_segment` - Пол (в текстовом виде);
- `os_name` - Тип операционной системы;
- `cpe_type_name` - Тип устройства;
- `country` - Страна проживания;
- `city` - Город проживания;
- `age_segment` - Возрастной сегмент;
- `traffic_segment` - Сегмент по объёму потребляемого трафика;
- `lifetime_segment` - Сегмент по количеству дней «жизни»;
- `nps_score` - Оценка клиента в NPS-опросе;
- `nps_group` - Поле хранит информацию о том, к какой группе относится оценка клиента в опросе;

In [1]:
#импортируем библиотеки
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [15]:
#подключаемся к базе
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local

elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform

else:
    raise Exception('Файл с базой данных SQLite не найден!')


if path_to_db:
  
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [16]:
#с помощью SQL запроса выводим таблицу в нужном виде
query = """
SELECT user_id,
       lt_day,
       CASE 
           WHEN lt_day <= 365 THEN 'New'
           WHEN lt_day > 365 THEN 'Old'
       END as is_new,
       age,
       CASE 
           WHEN gender_segment = 0.0 THEN 'M'
           WHEN gender_segment = 1.0 THEN 'F'
       END as gender_segment,
       os_name,
       cpe_type_name,
       l.country,
       l.city,
       SUBSTRING(age.title, 4) as age_segment,
       SUBSTRING(tr.title, 4) as traffic_segment,
       SUBSTRING(lt.title, 4) as lifetime_segment,
       nps_score,
       CASE
           WHEN nps_score >= 9 THEN 'promoters'
           WHEN nps_score = 7 OR nps_score = 8 THEN 'passives'
           WHEN nps_score <= 6 THEN 'detractors'
        END as nps_group
FROM user as u
LEFT JOIN location as l ON u.location_id = l.location_id
LEFT JOIN age_segment as age ON u.age_gr_id= age.age_gr_id
LEFT JOIN traffic_segment as tr ON u.tr_gr_id= tr.tr_gr_id
LEFT JOIN lifetime_segment as lt ON u.lt_gr_id= lt.lt_gr_id
WHERE gender_segment IS NOT NULL AND age IS NOT NULL
"""

In [17]:
df = pd.read_sql(query, engine)
df.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Old,45.0,F,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,Old,53.0,M,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,Old,57.0,M,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,Old,44.0,F,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,Old,24.0,M,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters


In [18]:
#сохраняем ДатаСет для создания дашборда
df.to_csv('telecomm.csv')

[Дашборд](https://public.tableau.com/app/profile/elizaveta3714/viz/NPS_16794341334890/sheet0?publish=yes)

[Презентация](https://disk.yandex.ru/i/C1gPwi-kf8ImOQ)